## A Demonstration of the RSA Cryptosystem

The following sequence of function calls to the `rsa` module (located in the [/src](https://github.com/dchampion/crypto/tree/master/code/src) directory of this repository) illustrates a session in which Alice signs and encrypts a private message, and sends the message to Bob over an insecure channel. Meanwhile, Eve&mdash;a passive adversary&mdash;intercepts all the information exchanged between Alice and Bob on the insecure channel.

It is assumed that Alice, Bob and Eve all possess their own copy of the `rsa` module; but that only Alice and Bob possess the signing and encryption keys produced by this module to achieve confidentiality, integrity and authentication of the private message Alice transmits to Bob.

Values below surrounded in square brackets [ ] are public (i.e., they can be transmitted over the insecure channel with no compromise to the security of the system, even if they are observed by Eve), and those that are not in square brackets are private (i.e., they must *not* be transmitted over the insecure channel).

The sequence begins with Alice generating her RSA key parameters using the `rsa` module's `generate_rsa_key` function, passing it a modulus size of `2048` bits. The function returns the quintuple `pA`, `qA`, `[nA]`, `d3A` and `d5A` (the suffix *A* on all the parameters identify them as belonging to Alice). Alice transmits her public modulus `[nA]` to Bob.

The `rsa` module defines the global constants `[3]` and `[5]` as the signature-verification and encryption keys, respectively, in order to minimize computational overhead (these are stored in the global variables `rsa.VERIFICATION_EXPONENT` and `rsa.ENCRYPTION_EXPONENT`). The small size of these values in no way compromises the security of the system; rather, the security of the system depends solely on the size of the public modulus (the bigger, the more secure). These constants, together with `[nA]`, comprise Alice's public key, and may be observed by Eve with no loss of security.

Alice must, however, keep the values `pA` and `qA`&mdash;the 1024-bit prime factors of `[nA]`&mdash;secret. She must also keep `d3A` and `d5A`&mdash;her signing and decryption keys, respectively&mdash;secret.

In [1]:
import sys
sys.path.append("../code/src")
import rsa
sys.path.append("../code/test")
import sym

pA, qA, nA, d3A, d5A = rsa.generate_rsa_key(2048)

To get an idea of the size and proportions of these parameters, let's print some of them out in decimal form. Note that `[nA]`, which is the product of `pA` and `qA`, is twice the length of each individually; `pA`'s and `qA`'s ~310 decimal digits to `[nA]`'s ~620.

In [2]:
print(pA)

137797313401830237271691148454204149826291279182572263925626859390771130435841299972887486938623795367181058316365464882373031062568738137223968332905137774170216481035313388335919217222212907080905585682642397969088403952466633511580399743988620520396229316942793027305240379370345674663532786879131572562447


In [3]:
print(qA)

136542630674835348671437596765204121826298559928728198256669091820683904787330675878583672827676610382124872042480794837517295175326183532215209418552708101423160167886913835889609502267898585627138080404842519295283818471037920566129817514934016974631167542512370015286167231167747855257344402022940392748567


In [4]:
print(nA)

18815207671810645446112202915048284595293025617977859527481999263937067794632138921400167875153003632868056272158847194641083418770519681844951962212444538375123203395642829060660508657831479362160159144356391623296330094210579391441740810424844147808403091848198317358154933401639998007001847584763150067106906391837402982732417801844742122052035876644531370750770284288459680741581553987561014479127482449247317963373733807615504974328246218894408730348600152486832685381735568000902431699665238633167928407987965266918750496324795657523073715033494586157394960797653560082212397595359503304013441603862537577263449


Recall that Alice requested a modulus of 2048 bits in length in her call to `generate_rsa_key`. Let's print the bit length of `[nA]` and see what we get.

In [5]:
print(nA.bit_length())

2048


What we see is that a string of 2048 bits equates to a decimal number of some 620 digits in length. We should see the same pattern if we print the bit lengths of `[nA]`'s prime factors `pA` and `qA`; that is, 1024 bits represents a decimal number of about 310 digits in length.

In [6]:
print(pA.bit_length())

1024


In [7]:
print(qA.bit_length())

1024


Now let's make sure `[nA]` is indeed the product of `pA` and `qA`. Note this step is not necessary for the protocol; it is done here just for demonstration purposes.

In [8]:
print(qA * pA == nA)

True


Now it's Bob's turn to generate RSA keys of his own. To differentiate Bob's keys from Alice's keys, we append Bob's with the captial letter *B*.

In [9]:
pB, qB, nB, d3B, d5B = rsa.generate_rsa_key(2048)

Because Bob requested a modulus size of 2048, based on our inspection of Alice's keys above we can conclude that Bob's keys are the same size as Alice's keys; so we don't need to print their values here.

Now that Alice and Bob have generated their keys, and transmitted to each other their public moduli `[nA]` and `[nB]`, they can proceed with a secure message exchange.

Let's say Alice wants to send Jenny's number (8675309) securely to Bob. First, Alice stores Jenny's number in the variable `mA`.

In [10]:
mA = "Sign and encrypt me!"

Then Alice calls the `rsa` module's `sign` fuction to sign the message, and stores the signature in the variable `[oA]`.

To sign the message, Alice needs to supply her secret signing key `d3A`, the two factors of `[nA]`, `pA` and `qA`, and the message `mA` to the rsa module's `sign` function.

In [11]:
oA = rsa.sign(d3A, pA, qA, mA)

Then Alice calls the `rsa` module's `encrypt_random_key` function, passing it Bob's RSA modulus `[nB]` and the global encryption exponent `[rsa.ENCRYPTION_EXPONENT]`, to compute a session key that will be used to encrypt the private message using a symmetric cipher (e.g., AES). This function returns the pair `KA` and `[cA]`, which are, respectively, the encryption key itself, and the ciphertext of a value Bob will use to replicate the same encryption key on his side of the channel.

Alice must keep the session key `KA` private, but she transmits the ciphertext `[cA]` to Bob.

In [12]:
KA, cA = rsa.encrypt_random_key(nB, rsa.ENCRYPTION_EXPONENT)

Then Alice encrypts the private message using a symmetric cipher.

In a real application, Alice and Bob would use an industrial-strength cipher (e.g., AES, 3DES) for message encryption. To keep things simple, however, we will use a bitwise exclusive-or (XOR) of the message with the symmetric key `KA` to serve as a cipher.

The result of this operation is the ciphertext `[mAC]`, from the XOR of `KA` with the message `mA`.

In [13]:
mAC = sym.encrypt(KA, mA)

Alice transmits the message signature `[oA]`, the message ciphertext `[mAC]` and the ciphertext of the symmetric-key's input material `[cA]` to Bob.

In order to decrypt the private message, Bob must reproduce the symmetric key Alice used to encrypt it. He does this by calling the `rsa` module's `decrypt_random_key` function, passing it his private decryption key `d5B`, the ciphertext of Alice's symmetric-key input material `[cA]`, and the two prime factors of his public modulus `pB` and `qB` (using `pB` and `qB`, instead of its equivalent `[nB]`, provides for faster decryption by means of the Chinese Remainder Theorem).

In [14]:
KB = rsa.decrypt_random_key(d5B, cA, pB, qB)

Let's make sure the key Bob decrypted above is the same as the key Alice encrypted. Again, this step is not required for the protocol; it is only used here to demonstrate that the keys are indeed equal.

In [15]:
print(KB == KA)

True


Now, Bob should be able to use the symmetric key `KB` to decrypt the message, by XOR'ing it with the ciphertext `mAC`. (An XOR is merely a bitwise toggle, in the first case from Alice's plaintext `mA` to the ciphertext `[mAC]`, and in the second case from `[mAC]` back to Bob's plaintext `mB` which, by this logic, should be identical to `mA`.

In [16]:
mB = sym.decrypt(KB, mAC)

If the assertion below holds, we know that Bob has successfully decrypted the private message encrypted, and sent to him, by Alice (i.e., Jenny's number).

In [17]:
print(mB == mA)

True


Let's print it here, just to prove that it is.

In [18]:
print(mB)

Sign and encrypt me!


All that remains to complete the session is for Bob to verify Alice's signature, and thereby prove that the message has not been altered and, further, that it originated from Alice (or, at least, that it was signed with Alice's private key).

He does this by calling the `rsa` module's `verify` fuction, passing Alice's public modulus `[nA]`, the constant signature-verification key `[rsa.VERIFICATION_EXPONENT]`, the decrypted message `mB` (Jenny's number) and the sigature sent to him by Alice `[oA]`.

In [19]:
verified = rsa.verify(nA, rsa.VERIFICATION_EXPONENT, mB, oA)

If the assertion below holds, we've proved that Bob has successfully decrypted and verified the message sent to him by Alice.

In [20]:
print(verified)

True
